In [5]:
import math
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from datetime import datetime
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

# Constants
MAX_TILT_ANGLE = 90  # Maximum tilt angle in degrees
SUNRISE = 6  # Sunrise time in hours (6 AM)
SUNSET = 18  # Sunset time in hours (6 PM)


# Functions to calculate solar position
def solar_declination(day_of_year: int) -> float:
    axial_tilt = 23.44  # Earth's axial tilt in degrees
    return axial_tilt * math.sin(math.radians((360 / 365) * (day_of_year - 81)))


def solar_elevation_angle(latitude: float, day_of_year: int, hour: int) -> float:
    declination = solar_declination(day_of_year)
    solar_time = hour - 12
    hour_angle = 15 * solar_time
    elevation_angle = math.degrees(
        math.asin(
            math.sin(math.radians(latitude)) * math.sin(math.radians(declination))
            + math.cos(math.radians(latitude))
            * math.cos(math.radians(declination))
            * math.cos(math.radians(hour_angle))
        )
    )
    return elevation_angle


# Function to update the plot based on the slider
def update_plot(hour, latitude, day_of_year):
    elevation_angle = solar_elevation_angle(latitude, day_of_year, hour)
    optimal_tilt_angle = max(0, min(elevation_angle, MAX_TILT_ANGLE))

    fig, ax = plt.subplots(figsize=(8, 6))

    # Solar panel
    panel_length = 1
    x = [0, panel_length * math.cos(math.radians(optimal_tilt_angle))]
    y = [0, panel_length * math.sin(math.radians(optimal_tilt_angle))]

    ax.plot(
        x, y, label=f"Solar Panel (Tilt Angle: {optimal_tilt_angle:.2f}°)", linewidth=5
    )

    # Horizon line
    ax.plot([-1, 1], [0, 0], "k--", linewidth=1)

    ax.set_xlim(-1, 1)
    ax.set_ylim(-0.1, 1.1)
    ax.set_aspect("equal", "box")
    ax.set_xlabel("Horizontal Distance")
    ax.set_ylabel("Vertical Distance")
    ax.set_title(
        f"Time of Day: {hour}:00\nSolar Elevation Angle: {elevation_angle:.2f}°"
    )
    ax.legend()
    ax.grid(True)
    plt.show()


# Example usage
latitude = 37.7749  # Latitude for San Francisco, CA
date = datetime.now()
day_of_year = date.timetuple().tm_yday

# Create the slider and interactive plot
hour_slider = widgets.IntSlider(
    value=12, min=SUNRISE, max=SUNSET, step=1, description="Hour:"
)
interact(
    update_plot,
    hour=hour_slider,
    latitude=fixed(latitude),
    day_of_year=fixed(day_of_year),
)